In [ ]:
import json
import os
import random
import glob

# --- Configuration ---
QA_CONTEXT_DIR = './QA_context' # Input directory for raw QA files
OUTPUT_DIR = './split_data' # Output directory for split files

TEST_SIZE = 40
VALID_SIZE = 50
# TRAIN_SIZE will be calculated dynamically

# --- Functions ---
def load_all_qa_data(base_dir):
    """Loads all QA pairs from .jsonl files in the specified directory."""
    all_data = []
    jsonl_files = glob.glob(os.path.join(base_dir, '**', '*.jsonl'), recursive=True)
    
    if not jsonl_files:
        print(f"경고: {base_dir} 경로에서 .jsonl 파일을 찾을 수 없습니다.")
        return []

    for file_path in jsonl_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line:
                    try:
                        all_data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"경고: {file_path} 파일에서 JSON 파싱 오류 발생: {e} - 줄: {line[:100]}...")
    return all_data

def write_jsonl(data, file_path):
    """Writes a list of dictionaries to a .jsonl file."""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"✅ {len(data)}개의 데이터가 {file_path} 에 저장되었습니다.")

# --- Main Execution ---
if __name__ == "__main__":
    print("데이터 분할 스크립트 시작...")

    # 1. 모든 QA 데이터 로드
    all_qa_data = load_all_qa_data(QA_CONTEXT_DIR)
    total_data_count = len(all_qa_data)
    print(f"총 {total_data_count}개의 QA 데이터를 로드했습니다.")

    if total_data_count == 0:
        print("로드된 데이터가 없어 분할을 진행할 수 없습니다.")
    elif total_data_count < TEST_SIZE + VALID_SIZE:
        print(f"경고: 총 데이터({total_data_count}개)가 테스트({TEST_SIZE}개) 및 검증({VALID_SIZE}개) 데이터 크기보다 작습니다. 분할 크기를 조정해주세요.")
    else:
        # 2. 데이터 셔플
        random.seed(42) # 재현성을 위해 시드 고정
        random.shuffle(all_qa_data)

        # 3. 데이터 분할
        test_data = all_qa_data[:TEST_SIZE]
        valid_data = all_qa_data[TEST_SIZE : TEST_SIZE + VALID_SIZE]
        train_data = all_qa_data[TEST_SIZE + VALID_SIZE :]

        # 4. 분할된 데이터 저장
        write_jsonl(test_data, os.path.join(OUTPUT_DIR, 'test.jsonl'))
        write_jsonl(valid_data, os.path.join(OUTPUT_DIR, 'validation.jsonl'))
        write_jsonl(train_data, os.path.join(OUTPUT_DIR, 'train.jsonl'))

        print("데이터 분할 및 저장 완료.")
        print(f"  - 훈련 데이터: {len(train_data)}개")
        print(f"  - 검증 데이터: {len(valid_data)}개")
        print(f"  - 테스트 데이터: {len(test_data)}개")


데이터 분할 스크립트 시작...
총 356개의 QA 데이터를 로드했습니다.
✅ 40개의 데이터가 ./text_data/test.jsonl 에 저장되었습니다.
✅ 50개의 데이터가 ./text_data/validation.jsonl 에 저장되었습니다.
✅ 266개의 데이터가 ./text_data/train.jsonl 에 저장되었습니다.
데이터 분할 및 저장 완료.
  - 훈련 데이터: 266개
  - 검증 데이터: 50개
  - 테스트 데이터: 40개
